## Imports

In [1]:
import mltlk
print(mltlk.__version__)
from mltlk import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten

0.1.13


2023-06-01 15:39:10.624150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load data
Load data, clean text and use Keras word vector embeddings preprocessing.

In [2]:
session = load_data("data/wikipedia_300.csv.gz", conf={
    "preprocess": "embeddings",
    "w2v_vector_size": 75,
    "stopwords": ["english", "stopwords/custom.csv"],
    "clean_text": "letters digits",
    "encode_labels": True,
    "max_length": 10000,
})

Info: Clean texts keeping letters and digits
Info: Labels encoded
Info: Load 180 stopwords from english, stopwords/custom.csv
Info: Vocabulary size is 53284
Info: 97.3% of sequences covered by max length 10000
Info: Loaded 300 examples in 2 categories


#### Show data stats

In [3]:
data_stats(session)

## Define Keras model
Builds the structure for the Keras model to use.

In [4]:
def get_model(session):
    model = Sequential()
    model.add(Embedding(input_dim=session["vocab_size"], output_dim=session["embeddings_size"], input_length=session["max_length"]))
    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation="softmax"))
    return model

## Evaluate model using train-test split
Build a RandomForest model and evaluate results using train-test split.

In [5]:
split_data(session, conf={
    "test_size": 0.1,
    "seed": 4,
    "stratify": True,
})

Info: Split data using 90% training data and 10% test data with seed 4 and stratify


In [6]:
evaluate_model(get_model(session), session, reload=False, conf={
    "mode": "split",
    "categories": True,
    "seed": 42,
    "epochs": 8,
    "batch_size": 32,
    "loss": "categorical_crossentropy",
})

Building and evaluating model using train-test split took 38.07 sec



Results,
Accuracy:,76.67%
F1-score:,76.64%
Precision:,76.79%
Recall:,76.67%


Games (0),80.00%,15
Programming (1),20.00%,3
Programming (1),73.33%,15
Games (0),26.67%,4


## Build final model and predict example
Build final model using all data and predict an unknown example.

In [7]:
build_model(get_model(session), session, conf={
    "seed": 42,
    "epochs": 8,
    "batch_size": 32,
    "loss": "categorical_crossentropy",
})
predict("This is an article about gamers - people who love playing games", session)

Info: Building final model on all data took 45.94 sec
Info: Example is predicted as Programming (1)
